In [1]:
# Importo los datos
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

# cargo los datos desde CSV a un panda común.
uyProp_df = pd.read_csv("uy_properties.csv")
# renombro las columnas que tienen nombres genéricos.
uyProp_df.rename(columns={"l1":"Pais"}, inplace=True)
uyProp_df.rename(columns={"l2":"Dpto"}, inplace=True)
uyProp_df.rename(columns={"l3":"Localidad/Barrio"}, inplace=True)
uyProp_df.rename(columns={"l4":"Zona"}, inplace=True)
# elimino las columnas que ya se que son 100% datos null.
uyProp_df.drop("l5", axis="columns", inplace=True)
uyProp_df.drop("l6", axis="columns", inplace=True)
# elimino las filas que contengan datos de locación nulos o incompletos.
uyProp_df.dropna(subset=['lon', 'lat', 'surface_total', 'surface_covered', 'price'], inplace=True)
# por motivos de performance me quedo con las 1000 primeras filas.
# uyProp1000_df = uyProp_df.head(1000)

uyProp_df.shape
# list(uyProp_df.columns)
# 

(1008, 23)

## Mapa básico con un marcador por propiedad.
#### El pop-up de cada marcador es el título de la publicación.

In [7]:
import folium
from folium.plugins import MarkerCluster

# Creo un mapa sencillo con las coordenadas de Uruguay.
mapa = folium.Map(location=[-32.5583168, -55.811697],
                 zoom_start=7)
mapaHeat = folium.Map(location=[-32.5583168, -55.811697],
                     zoom_start=7)
mapaClusteres = folium.Map(location=[-32.5583168, -55.811697],
                            zoom_start=7)

# Creo un objeto cluster de marcadores para el mapa de clusters de marcadores y lo agrego al mapa.
cluster_marcadores = MarkerCluster().add_to(mapaClusteres)

# Muestro un marcador en el mapa por cada propiedad listada en el dataframe.
for index, row in uyProp_df.iterrows():
    folium.Marker([row['lat'], row['lon']], popup=row['title']).add_to(mapa)
    folium.Marker([row['lat'], row['lon']], popup=row['title']).add_to(cluster_marcadores)

# Muestro el mapa.
mapa

#### Mismo mapa pero con marcadores clusterizados para más cómoda visibilidad.

In [9]:
# Muestro el mapa de clusteres de marcadores.
mapaClusteres

## Mapa de calor de concentración de propiedades en el mapa

In [4]:
# HEATMAP de concentración de las propiedades en el mapa.
from folium.plugins import HeatMap

# Achico la información a manejar, para una mejor performance
uyPropHeat_df = uyProp_df[["lat", "lon"]]
# Me aseguro de trabajar latitud y longitud como float.
uyPropHeat_df["lat"] = uyPropHeat_df["lat"].astype(float)
uyPropHeat_df["lon"] = uyPropHeat_df["lon"].astype(float)

# Genero una lista de listas para input en la capa del HeatMap
heat_data = [[row["lat"],row["lon"]] for index, row in uyPropHeat_df.iterrows()]

# Genero la capa del mapa de calor
HeatMap(heat_data).add_to(mapaHeat)

mapaHeat